# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

### 2. Load the dataset

In [2]:
df = pd.read_csv('car+data.csv')
df.sample(5)

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
44,omni,2012,1.25,2.69,50000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
219,verna,2012,4.50,9.40,36000,Petrol,Dealer,Manual,0
173,Activa 4g,2017,0.40,0.51,1300,Petrol,Individual,Automatic,0
22,sx4,2011,4.40,8.01,50000,Petrol,Dealer,Automatic,0


### 3. Check the shape and basic information of the dataset.

In [3]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [4]:
duplicates = df.duplicated()

# If duplicates are found, drop them
if duplicates.any():
    df.drop_duplicates(inplace=True)
    print("Duplicate records have been dropped.")
else:
    print("No duplicate records found.")

Duplicate records have been dropped.


### 5. Drop the columns which you think redundant for the analysis.

In [5]:
redundant_columns = ['Car_Name']
df.drop(redundant_columns, axis=1, inplace=True)


### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [6]:
import datetime
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
current_year = datetime.datetime.now().year
df['age_of_the_car'] = current_year - df['Year'].dt.year
df.drop('Year', axis=1, inplace=True)

### 7. Encode the categorical columns

In [7]:
from sklearn.preprocessing import LabelEncoder
categorical_columns = ['Fuel_Type', 'Seller_Type', 'Transmission']
label_encoder = LabelEncoder()
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

### 8. Separate the target and independent features.

In [8]:
X = df.drop('Selling_Price',axis=1)
y = df['Selling_Price']

### 9. Split the data into train and test.

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(209, 7) (90, 7)
(209,) (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [10]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train,y_train)

RandomForestRegressor()

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

r2_train = r2_score(y_train,y_train_pred)
r2_test = r2_score(y_test,y_test_pred)

print('r2 train:',r2_train)
print('r2 test',r2_test)

r2 train: 0.9787232609476814
r2 test 0.8978979501228657


### 11. Create a pickle file with an extension as .pkl

In [13]:
import pickle
pickle.dump(rf, open('mode.pkl','wb'))
model = pickle.load(open('mode.pkl','rb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [14]:
<!DOCTYPE html>
<html>
<head>
    <title>Car Price Prediction</title>
    <style>
        body {
            font-family: Arial, sans-serif;
        }

        h1 {
            text-align: center;
        }

        form {
            width: 400px;
            margin: 0 auto;
        }

        label {
            display: block;
            margin-bottom: 10px;
            font-weight: bold;
        }

        input[type="text"],
        select {
            width: 100%;
            padding: 5px;
            margin-bottom: 10px;
        }

        input[type="submit"] {
            width: 100%;
            padding: 10px;
            background-color: #4CAF50;
            color: white;
            font-weight: bold;
            border: none;
            cursor: pointer;
        }

        #result {
            margin-top: 20px;
            text-align: center;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <h1>Car Price Prediction</h1>
    <form id="carForm">
        <label for="age">Age of the Car:</label>
        <input type="text" id="age" name="age" required><br><br>

        <label for="fuelType">Fuel Type:</label>
        <select id="fuelType" name="fuelType" required>
            {% for fuelType in fuelTypes %}
                <option value="{{ fuelType }}">{{ fuelType }}</option>
            {% endfor %}
        </select><br><br>

        <label for="sellerType">Seller Type:</label>
        <select id="sellerType" name="sellerType" required>
            {% for sellerType in sellerTypes %}
                <option value="{{ sellerType }}">{{ sellerType }}</option>
            {% endfor %}
        </select><br><br>

        <label for="transmission">Transmission:</label>
        <select id="transmission" name="transmission" required>
            {% for transmission in transmissions %}
                <option value="{{ transmission }}">{{ transmission }}</option>
            {% endfor %}
        </select><br><br>

        <label for="owner">Owner:</label>
        <select id="owner" name="owner" required>
            {% for owner in owners %}
                <option value="{{ owner }}">{{ owner }}</option>
            {% endfor %}
        </select><br><br>

        <label for="kmsDriven">Kilometers Driven:</label>
        <input type="text" id="kmsDriven" name="kmsDriven" required><br><br>

        <input type="submit" value="Submit">
    </form>

    <div id="result"></div>

    <script>
    document.getElementById("carForm").addEventListener("submit", function(e) {
        e.preventDefault(); // Prevent form submission
        const formData = new FormData(this);

        // Convert form data to JSON
        const jsonData = {};
        for (const [key, value] of formData.entries()) {
            jsonData[key] = value;
        }

        // Send the JSON data to the API endpoint
        fetch('/predict', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify(jsonData)
        })
        .then(response => response.json())
        .then(data => {
            // Display the predicted price
            document.getElementById("result").innerText = "Predicted Price: " + data.predicted_price;
        })
        .catch(error => {
            console.error('Error:', error);
        });
    });
    </script>

</body>
</html>
     
     


SyntaxError: invalid syntax (1297222688.py, line 1)

### b) Create app.py file and write the predict function

In [ ]:
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')
import joblib
from flask import Flask, request, render_template, jsonify
import pandas as pd

app = Flask(__name__)  # Initialize the flask App
model = joblib.load('model.pkl')  # loading the trained model

fuelTypes = ['Petrol', 'Diesel', 'CNG']
sellerTypes = ['Dealer', 'Individual']
transmissions = ['Manual', 'Automatic']
owners = ['0', '1', '3']


@app.route('/')  # Homepage
def index():
    return render_template('index.html', fuelTypes=fuelTypes, sellerTypes=sellerTypes, transmissions=transmissions,
                           owners=owners)


@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    age = int(data['age'])
    fuelType = data['fuelType']
    sellerType = data['sellerType']
    transmission = data['transmission']
    owner = int(data['owner'])

    input_data = pd.DataFrame({
        'AGE': [age],
        'Fuel_Types': [fuelType],
        'Seller_Type': [sellerType],
        'Transmission': [transmission],
        'Owner': [owner]
    })
    input_data = pd.get_dummies(input_data, drop_first=True)
    predicted_price = model.predict(input_data)[0]
    response = {'predicted_price': predicted_price}
    return jsonify(response)


if __name__ == "__main__":
    app.run(debug=True)


### 13. Deploy your app on Heroku. (write commands for deployment)

### 14. Paste the URL of the heroku application below, and while submitting the solution submit this notebook along with the source code.

### Happy Learning :)